<a href="https://colab.research.google.com/github/santiagonajera/PlanificacionProduccion/blob/main/Clase1_2octubre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from io import BytesIO

# URL del archivo Excel
url = "https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Historico_Ventas_Con_Precio_Costo.xlsx"

# Descargar el archivo
response = requests.get(url)
if response.status_code != 200:
    raise Exception("Error al descargar el archivo")

# Cargar el archivo Excel desde memoria
excel_data = BytesIO(response.content)
df = pd.read_excel(excel_data, sheet_name='Ventas')

# Verificar estructura
print("Primeras filas del dataset:")
print(df.head())

# Asumimos que la primera columna es el índice (meses) y el resto son los items
# Eliminamos la primera columna si es texto (meses) y usamos solo las columnas numéricas
# Si la primera columna no tiene nombre o se llama 'Unnamed', la ignoramos
# Pero en este caso, parece que la primera columna es el índice (meses)

# Seleccionamos solo las columnas numéricas (todos los items)
item_columns = df.columns[1:]  # Desde la segunda columna en adelante

# Calcular media y desviación estándar para cada item
means = df[item_columns].mean()
stds = df[item_columns].std()

# Calcular coeficiente de variación (en porcentaje)
cv = (stds / means) * 100

# Crear un DataFrame con los resultados
cv_results = pd.DataFrame({
    'Item': item_columns,
    'Media': means.values,
    'Desviación Estándar': stds.values,
    'Coeficiente de Variación (%)': cv.values
}).round(2)

# Mostrar resultados
print("\nCoeficientes de Variación por Item:")
print(cv_results.to_string(index=False))

# Opcional: guardar resultados en un archivo CSV
cv_results.to_csv('coeficientes_variacion_items.csv', index=False)
print("\n✅ Resultados guardados en 'coeficientes_variacion_items.csv'")

Primeras filas del dataset:
  Unnamed: 0  Item_1  Item_2  Item_3  Item_4  Item_5  Item_6  Item_7  Item_8  \
0 2023-01-01     198     177     162     181     185     199     157      80   
1 2023-02-01     200     204     180     178      65     210     153      85   
2 2023-03-01     216     192     180     196     201     228     176      85   
3 2023-04-01     231     213     176     204     231     212     185     118   
4 2023-05-01     219     197     190     205     212     211     196     107   

   Item_9  ...  Item_41  Item_42  Item_43  Item_44  Item_45  Item_46  Item_47  \
0     157  ...      200       67      159      197      181      200       49   
1     155  ...      211       61      169      205      213      186       85   
2     191  ...      212       49      161      218      199      191       86   
3     166  ...      223       76      159      210      182      200       86   
4     180  ...      223       91      176      210      212      234       83   

   I

In [2]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime

# 1. Descargar y cargar los datos
url = "https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Historico_Ventas_Con_Precio_Costo.xlsx"
response = requests.get(url)
response.raise_for_status()  # Lanza un error si la descarga falla

df = pd.read_excel(BytesIO(response.content), sheet_name='Ventas')

# 2. Preparar el índice de fechas
# La primera columna contiene los meses en formato 'ene-2023', etc.
# Convertimos a datetime para poder trabajar con fechas
def parse_month_year(date_str):
    # Mapeo de abreviaturas en español a inglés
    spanish_to_english = {
        'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr',
        'may': 'May', 'jun': 'Jun', 'jul': 'Jul', 'ago': 'Aug',
        'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
    }
    for es, en in spanish_to_english.items():
        date_str = date_str.replace(es, en)
    return pd.to_datetime(date_str, format='%b-%Y')

# Aplicar la conversión a la primera columna
date_col = df.columns[0]
df[date_col] = df[date_col].astype(str).apply(parse_month_year)
df.set_index(date_col, inplace=True)

# 3. Seleccionar solo las columnas de items (asumiendo que son todas las demás)
item_columns = df.columns

# 4. Crear un DataFrame para el pronóstico que incluya todo el rango hasta dic-2026
last_date = df.index[-1]  # Última fecha en los datos históricos (may-2024)
forecast_end = pd.to_datetime('2026-12-01')
all_dates = pd.date_range(start=df.index[0], end=forecast_end, freq='MS')  # 'MS' = mes start

# Crear un DataFrame vacío con todas las fechas
df_full = pd.DataFrame(index=all_dates, columns=item_columns, dtype='float')

# Llenar con los datos históricos conocidos
df_full.loc[df.index, :] = df.values

# 5. Aplicar el pronóstico con promedio móvil simple de 2 meses
for col in item_columns:
    for i in range(len(df_full)):
        if pd.isna(df_full.iloc[i][col]):
            # Tomar los dos meses anteriores (si existen)
            prev_values = df_full[col].iloc[max(0, i-2):i].dropna()
            if len(prev_values) == 2:
                forecast_val = prev_values.mean()
                df_full.iloc[i, df_full.columns.get_loc(col)] = forecast_val
            else:
                # Si no hay suficientes datos, no se puede pronosticar
                break

# 6. Filtrar solo el período de pronóstico (desde jun-2024 hasta dic-2026)
forecast_start = pd.to_datetime('2024-06-01')
forecast_df = df_full.loc[forecast_start:forecast_end].copy()

# Formatear el índice para que se vea como 'jun-2024', etc.
def format_date_to_spanish(dt):
    month_names = {
        1: 'ene', 2: 'feb', 3: 'mar', 4: 'abr',
        5: 'may', 6: 'jun', 7: 'jul', 8: 'ago',
        9: 'sep', 10: 'oct', 11: 'nov', 12: 'dic'
    }
    return f"{month_names[dt.month]}-{dt.year}"

forecast_df.index = forecast_df.index.to_series().apply(format_date_to_spanish)

# 7. Guardar resultados
forecast_df.to_excel('forecast_2026_promedio_movil_2meses.xlsx')
print("✅ Pronóstico generado y guardado en 'forecast_2026_promedio_movil_2meses.xlsx'")

# Opcional: mostrar las primeras filas del pronóstico
print("\nPronóstico (primeros 6 meses):")
print(forecast_df.head(6))

ValueError: time data "2023-01-01" doesn't match format "%b-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [3]:
import pandas as pd

# Cargar el archivo de Excel
ruta_archivo = 'Historico_Ventas_Con_Precio_Costo.xlsx'
hoja = 'Ventas'
df = pd.read_excel(ruta_archivo, sheet_name=hoja)

# Transponer los datos para tener los meses como filas y los items como columnas
df = df.set_index(df.columns[0]).T

# Función para calcular promedios móviles y proyección
def proyectar_ventas(df, ventana=3):
    proyecciones = {}
    for item in df.columns:
        # Calcular promedios móviles
        df[f'{item}_promedio_movil'] = df[item].rolling(window=ventana).mean()

        # Proyección para 2026 (12 meses)
        ultimo_promedio = df[f'{item}_promedio_movil'].iloc[-1]
        proyeccion = [ultimo_promedio] * 12
        proyecciones[item] = proyeccion

    return pd.DataFrame(proyecciones, index=[f'mes-{i+1}-2026' for i in range(12)])

# Calcular proyección
proyeccion_2026 = proyectar_ventas(df)

# Guardar proyección en un nuevo archivo de Excel
proyeccion_2026.to_excel('Proyeccion_Ventas_2026.xlsx', index_label='Mes')

print("Proyección de ventas para 2026 guardada en 'Proyeccion_Ventas_2026.xlsx'")


FileNotFoundError: [Errno 2] No such file or directory: 'Historico_Ventas_Con_Precio_Costo.xlsx'

In [4]:
import pandas as pd
import requests
from io import BytesIO

# URL del archivo en GitHub
url = "https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Historico_Ventas_Con_Precio_Costo.xlsx"

# Descargar el archivo desde GitHub
response = requests.get(url)
if response.status_code == 200:
    # Leer el archivo descargado con pandas
    df = pd.read_excel(BytesIO(response.content), sheet_name='Ventas')

    # Transponer los datos para tener los meses como filas y los items como columnas
    df = df.set_index(df.columns[0]).T

    # Función para calcular promedios móviles y proyección
    def proyectar_ventas(df, ventana=3):
        proyecciones = {}
        for item in df.columns:
            # Calcular promedios móviles
            df[f'{item}_promedio_movil'] = df[item].rolling(window=ventana).mean()

            # Proyección para 2026 (12 meses)
            ultimo_promedio = df[f'{item}_promedio_movil'].iloc[-1]
            proyeccion = [ultimo_promedio] * 12
            proyecciones[item] = proyeccion

        return pd.DataFrame(proyecciones, index=[f'mes-{i+1}-2026' for i in range(12)])

    # Calcular proyección
    proyeccion_2026 = proyectar_ventas(df)

    # Guardar proyección en un nuevo archivo de Excel
    proyeccion_2026.to_excel('Proyeccion_Ventas_2026.xlsx', index_label='Mes')
    print("Proyección de ventas para 2026 guardada en 'Proyeccion_Ventas_2026.xlsx'")
else:
    print("No se pudo descargar el archivo desde GitHub.")


Proyección de ventas para 2026 guardada en 'Proyeccion_Ventas_2026.xlsx'


In [5]:
import pandas as pd
import requests
from io import BytesIO

# URL del archivo en GitHub
url = "https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Historico_Ventas_Con_Precio_Costo.xlsx"

# Descargar el archivo desde GitHub
response = requests.get(url)
if response.status_code == 200:
    # Leer el archivo descargado con pandas
    df = pd.read_excel(BytesIO(response.content), sheet_name='Ventas')

    # Transponer los datos para tener los meses como filas y los items como columnas
    df = df.set_index(df.columns[0]).T

    # Función para calcular promedios móviles y proyección
    def proyectar_ventas(df, ventana=3):
        proyecciones = {}
        for item in df.columns:
            # Calcular promedios móviles
            df[f'{item}_promedio_movil'] = df[item].rolling(window=ventana).mean()

            # Proyección para 2026 (12 meses)
            ultimo_promedio = df[f'{item}_promedio_movil'].iloc[-1]
            proyeccion = [ultimo_promedio] * 12
            proyecciones[item] = proyeccion

        return pd.DataFrame(proyecciones, index=[f'mes-{i+1}-2026' for i in range(12)])

    # Calcular proyección
    proyeccion_2026 = proyectar_ventas(df)

    # Imprimir el DataFrame de proyección
    print("Proyección de ventas para 2026 (promedio móvil de los últimos 3 meses):")
    print(proyeccion_2026)
else:
    print("No se pudo descargar el archivo desde GitHub.")


Proyección de ventas para 2026 (promedio móvil de los últimos 3 meses):
             2023-01-01  2023-02-01  2023-03-01  2023-04-01  2023-05-01  \
mes-1-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-2-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-3-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-4-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-5-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-6-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-7-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-8-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-9-2026   135.333333       143.0  154.666667  157.666667  158.666667   
mes-10-2026  135.333333       143.0  154.666667  157.666667  158.666667   
mes-11-2026  135.333333       143.0  154.666667  157.666667  158.666667   
mes-12-2026  135.333333     

In [6]:
import pandas as pd
import requests
from io import BytesIO

# URL del archivo en GitHub
url = "https://github.com/santiagonajera/PlanificacionProduccion/raw/refs/heads/main/Historico_Ventas_Con_Precio_Costo.xlsx"

# Descargar el archivo desde GitHub
response = requests.get(url)
if response.status_code == 200:
    # Leer el archivo descargado con pandas
    df = pd.read_excel(BytesIO(response.content), sheet_name='Ventas')

    # Los nombres de los items están en la primera fila, así que usamos header=0
    df = df.set_index(df.columns[0])

    # Función para calcular promedios móviles y proyección
    def proyectar_ventas(df, ventana=3):
        proyecciones = {}
        for item in df.columns:
            # Calcular promedios móviles
            df[f'{item}_promedio_movil'] = df[item].rolling(window=ventana).mean()

            # Proyección para 2026 (12 meses)
            ultimo_promedio = df[f'{item}_promedio_movil'].iloc[-1]
            proyeccion = [ultimo_promedio] * 12
            proyecciones[item] = proyeccion

        return pd.DataFrame(proyecciones, index=[f'mes-{i+1}-2026' for i in range(12)])

    # Calcular proyección
    proyeccion_2026 = proyectar_ventas(df)

    # Imprimir el DataFrame de proyección
    print("Proyección de ventas para 2026 (promedio móvil de los últimos 3 meses):")
    print(proyeccion_2026)
else:
    print("No se pudo descargar el archivo desde GitHub.")


Proyección de ventas para 2026 (promedio móvil de los últimos 3 meses):
                 Item_1      Item_2      Item_3  Item_4      Item_5  \
mes-1-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-2-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-3-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-4-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-5-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-6-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-7-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-8-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-9-2026   279.333333  273.333333  210.333333   271.0  249.666667   
mes-10-2026  279.333333  273.333333  210.333333   271.0  249.666667   
mes-11-2026  279.333333  273.333333  210.333333   271.0  249.666667   
mes-12-2026  279.333333  273.333333  210.333333   271.0  249.666667   

    